# 可视化开发环境配置

***注：此配置步骤在新开通的命名空间下只需要执行一次即可***

## 准备工作运行时的必要文件
执行下面的代码，准备可视化工作里运行时需要的文件

In [1]:
import boto3
import json
import os

s3 = boto3.resource(
    "s3", endpoint_url="http://minio-service.kubeflow:9000", aws_access_key_id="minio", aws_secret_access_key="minio123"
)

src_dir = "src/elyra-scripts-3.12.0"
bucket = s3.Bucket("public")
if not bucket.creation_date:
    bucket = s3.create_bucket(Bucket="public")

bucket_name = "public"
bucket_policy = {
    "Version": "2012-10-17",
    "Statement": [
        {
            "Sid": "AddPerm",
            "Effect": "Allow",
            "Principal": "*",
            "Action": ["s3:GetObject"],
            "Resource": f"arn:aws:s3:::{bucket_name}/*",
        }
    ],
}

# Convert the policy from JSON dict to string
bucket_policy = json.dumps(bucket_policy)

# Set the new policy
s3_client = s3.meta.client
s3_client.put_bucket_policy(Bucket=bucket_name, Policy=bucket_policy)

for fname in os.listdir(src_dir):
    src = os.path.join(src_dir, fname)
    print("uploading ", fname)
    s3_client.upload_file(src, "public", fname)
print("done")

uploading  bootstrapper.py
uploading  pip.conf
uploading  requirements-elyra-py37.txt
uploading  requirements-elyra.txt
done


## 配置可视化任务提交地址

使用下面命令直接创建，或者参考文档：https://elyra.readthedocs.io/en/latest/user_guide/runtime-conf.html 图形化的创建

***注意：`--user_namespace kubeflow-admin-cpaas-io`根据当前用户的命名空间配置，`cos_username`, `cos_password`根据 S3 的账号密码配置***

In [1]:
%%bash
elyra-metadata create runtimes \
       --schema_name kfp \
       --display_name "kubeflow" \
       --api_endpoint http://ml-pipeline.kubeflow:8888 \
       --user_namespace kubeflow-admin-cpaas-io \
       --auth_type "KUBERNETES_SERVICE_ACCOUNT_TOKEN" \
       --engine Argo \
       --cos_endpoint http://minio-service.kubeflow:9000 \
       --cos_auth_type "USER_CREDENTIALS" \
       --cos_username minio \
       --cos_password minio123 \
       --cos_bucket mlpipeline \
       --tags "['kfp', 'v1.0']"

[I 2023-01-10 02:40:43,172.172] Upgrading runtime kfp instance 'kubeflow' to include runtime_type 'KUBEFLOW_PIPELINES'...


Metadata instance 'kubeflow' for schema 'kfp' has been written to: /home/jovyan/.local/share/jupyter/metadata/runtimes/kubeflow.json


## 配置任务运行时镜像

使用下面命令直接创建，或者参考文档：https://elyra.readthedocs.io/en/latest/user_guide/runtime-image-conf.html 图形化的创建

In [2]:
%%bash
elyra-metadata create runtime-images \
    --name "mlops-rt-tensorflow-250" \
    --display_name "mlops-rt-tensorflow-250" \
    --description "Alauda MLOps runtime image with tensorflow 2.5.0" \
    --image_name "build-harbor.alauda.cn/3rdparty/kubeflow/elyra-runtime-tensorflow:2.5.0" && \
elyra-metadata create runtime-images \
    --name "mlops-rt-intel-tensorflow-250" \
    --display_name "mlops-rt-intel-tensorflow-2.5.0" \
    --description "Alauda MLOps runtime image with intel tensorflow 2.5.0" \
    --image_name "build-harbor.alauda.cn/3rdparty/kubeflow/elyra-runtime-intel-tensorflow:2.5.0" && \
elyra-metadata create runtime-images \
    --name "mlops-rt-kaniko" \
    --display_name "mlops-rt-kaniko" \
    --description "Alauda MLOps runtime image kaniko image builder" \
    --image_name "build-harbor.alauda.cn/3rdparty/kubeflow/elyra-kaniko:latest" && \
elyra-metadata create runtime-images \
    --name "mlops-rt-tensorflow-2100" \
    --display_name "mlops-rt-tensorflow-2100" \
    --description "Alauda MLOps runtime image with tensorflow 2.10.0" \
    --image_name "build-harbor.alauda.cn/3rdparty/kubeflow/elyra-runtime-tensorflow:2.10.0" && \
elyra-metadata create runtime-images \
    --name "mlops-rt-pytorch" \
    --display_name "mlops-rt-pytorch" \
    --description "Alauda MLOps runtime image with pytorch 1.13.0" \
    --image_name "build-harbor.alauda.cn/3rdparty/kubeflow/elyra-runtime-pytorch:1.13.0"

Metadata instance 'mlops-rt-tensorflow-250' for schema 'runtime-image' has been written to: /home/jovyan/.local/share/jupyter/metadata/runtime-images/mlops-rt-tensorflow-250.json
Metadata instance 'mlops-rt-intel-tensorflow-250' for schema 'runtime-image' has been written to: /home/jovyan/.local/share/jupyter/metadata/runtime-images/mlops-rt-intel-tensorflow-250.json
Metadata instance 'mlops-rt-kaniko' for schema 'runtime-image' has been written to: /home/jovyan/.local/share/jupyter/metadata/runtime-images/mlops-rt-kaniko.json
